# SignalFlow Business Domain Analysis

This notebook demonstrates how to use the SignalFlow SDK for exploring meeting data,
analyzing signals, and generating AI-powered insights.

## Setup

First, let's import the SDK and connect to the environment.

In [ ]:
# Add the project root to the path
import sys
sys.path.insert(0, '..')

# Import the SDK
from src.app.sdk import SignalFlowClient

# Connect to staging environment (default)
# Options: 'local', 'staging', 'production'
client = SignalFlowClient(environment="staging")

# Verify connection
print(f"Connected to: {client.environment.name}")
print(f"API URL: {client.environment.api_url}")

## 1. Exploring Meetings

The `MeetingsExplorer` provides methods to list, search, and analyze meetings.

In [ ]:
# List recent meetings
meetings = client.meetings.list(limit=20)
print(f"Found {len(meetings)} meetings\n")

# Display meeting names
for m in meetings[:5]:
    print(f"- {m.meeting_name} ({m.meeting_date or 'No date'})")

In [ ]:
# Convert to DataFrame for analysis
import pandas as pd

df_meetings = client.meetings.to_dataframe()
df_meetings.head()

In [ ]:
# Get meeting statistics
stats = client.meetings.get_statistics()
print("Meeting Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

## 2. Analyzing Signals

Signals are extracted from meetings: decisions, actions, blockers, risks, and ideas.

In [ ]:
# List all signals
signals = client.signals.list(limit=50)
print(f"Found {len(signals)} signals\n")

# Show signal distribution
signal_counts = client.signals.get_by_type()
print("Signals by Type:")
for sig_type, count in signal_counts.items():
    print(f"  {sig_type}: {count}")

In [ ]:
# Filter by signal type
actions = client.signals.list(signal_type="action", limit=10)
print("Recent Actions:")
for a in actions[:5]:
    status = '✅' if a.status == 'completed' else '⏳'
    print(f"{status} {a.content[:60]}...")

In [ ]:
# Search signals semantically
results = client.signals.search("architecture decisions")
print(f"Search found {len(results)} signals")
for r in results[:3]:
    print(f"- [{r.signal_type}] {r.content[:80]}...")

## 3. DIKW Knowledge Hierarchy

Explore the Data → Information → Knowledge → Wisdom pipeline.

In [ ]:
# Get DIKW distribution
dikw_counts = client.dikw.get_by_level()
print("DIKW Distribution:")
for level, count in dikw_counts.items():
    bar = '█' * min(count, 20)
    print(f"  {level:12}: {bar} {count}")

In [ ]:
# List knowledge items
knowledge = client.dikw.list(level="knowledge", limit=10)
print("Knowledge Items:")
for item in knowledge:
    print(f"- {item.get('title', 'Untitled')}")

## 4. Career Development

Track skills, standups, and career progression.

In [ ]:
# Get standups
standups = client.career.get_standups(limit=5)
print("Recent Standups:")
for s in standups:
    print(f"- {s.get('created_at', '')[:10]}: {s.get('update_text', '')[:60]}...")

In [ ]:
# Get skills
skills_df = client.career.to_dataframe("skills")
if len(skills_df) > 0:
    skills_df.head()

## 5. AI Insights

Generate summaries, trends, and recommendations.

In [ ]:
# Weekly summary
summary = client.insights.summarize_week()
print(f"Week: {summary.start_date} to {summary.end_date}")
print(f"Meetings: {summary.meeting_count}")
print(f"Open Actions: {summary.open_actions}")
print(f"Completed Actions: {summary.completed_actions}")
print(f"\nTop Themes: {', '.join(summary.top_themes[:5])}")

In [ ]:
# Trend analysis
trend = client.insights.trend_analysis("meeting_count", days=14)
print(f"Meeting trend ({trend.period}): {trend.trend_direction}")
print(f"Change: {trend.change_percent}%")

In [ ]:
# Get recommendations
recs = client.insights.recommendations()
print("Recommendations:")
for r in recs:
    priority = {'high': '🔴', 'medium': '🟡', 'low': '🟢'}.get(r['priority'], '⚪')
    print(f"{priority} [{r['type']}] {r['description']}")

## 6. Direct Supabase Access

For advanced queries, you can access the Supabase client directly.

In [ ]:
# Direct Supabase query (requires SUPABASE_URL and SUPABASE_KEY env vars)
if client.supabase:
    result = client.supabase.table("meetings").select("id, meeting_name").limit(5).execute()
    for row in result.data:
        print(row)
else:
    print("Supabase client not available - using API only mode")

## 7. Visualization Examples

In [ ]:
import matplotlib.pyplot as plt

# Signal type distribution chart
signal_counts = client.signals.get_by_type()

if signal_counts:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(signal_counts.keys(), signal_counts.values(), color=['#4CAF50', '#2196F3', '#FF9800', '#f44336', '#9C27B0'])
    ax.set_xlabel('Signal Type')
    ax.set_ylabel('Count')
    ax.set_title('Signal Distribution by Type')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No signal data to visualize")

In [ ]:
# DIKW pyramid visualization
dikw_counts = client.dikw.get_by_level()
levels = ['data', 'information', 'knowledge', 'wisdom']
values = [dikw_counts.get(l, 0) for l in levels]

if sum(values) > 0:
    fig, ax = plt.subplots(figsize=(8, 6))
    colors = ['#E3F2FD', '#90CAF9', '#42A5F5', '#1565C0']
    ax.barh(levels, values, color=colors)
    ax.set_xlabel('Count')
    ax.set_title('DIKW Knowledge Pyramid')
    plt.tight_layout()
    plt.show()
else:
    print("No DIKW data to visualize")

## 8. Health Check & Diagnostics

In [ ]:
# Check system health
health = client.health_check()
print(f"API Status: {'✅ Healthy' if health.get('status') == 'healthy' else '❌ Unhealthy'}")
print(f"Version: {health.get('version', 'unknown')}")

In [ ]:
# Run diagnostics
diag = client.diagnostics()
print("System Diagnostics:")
for key, value in diag.items():
    print(f"  {key}: {value}")